### starting a basic sequence to sequence LSTM model 

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')


2024-06-03 07:39:57.401260: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-03 07:39:57.670620: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 07:39:58.509523: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# creating a sintetic dataframe

In [2]:
num_days = 365


start_date = datetime(2023, 1, 1)


dates = [start_date + timedelta(days=i) for i in range(num_days)]


sales = np.random.randint(10, 100, size=num_days)
sales1 = np.random.randint(9, 20, size=num_days)

df = pd.DataFrame({'data': dates, 'vendas': sales,'valor': sales})

df = df.reset_index().rename(columns={"index":"time_idx"})

df = df[['time_idx','vendas','valor']]
df.head()

,time_idx,vendas,valor
0,0,64,64
1,1,86,86
2,2,51,51
3,3,11,11
4,4,96,96


In [3]:
for col in df.select_dtypes(include=['int']):
    df[col] = df[col].astype(float)

In [4]:
datas = df.time_idx
X = df.iloc[:,1:]

In [5]:
X

,vendas,valor
0,64.0,64.0
1,86.0,86.0
2,51.0,51.0
3,11.0,11.0
4,96.0,96.0
...,...,...
360,69.0,69.0
361,47.0,47.0
362,98.0,98.0
363,56.0,56.0


In [6]:
X.shape

(365, 2)

In [7]:
datas = np.expand_dims(datas, axis=0)
datas = np.expand_dims(datas, axis=-1)
datas.shape

(1, 365, 1)

In [8]:
X = np.expand_dims(X, axis=0)
X.shape

(1, 365, 2)

In [9]:
lstm_encoder = tf.keras.layers.LSTM(units=64,
                            return_state = True,
                            name = 'LSTM_encoder')

lstm_decoder = tf.keras.layers.LSTM(units=64,
                             return_sequences=True, 
                             return_state=True,
                             name = 'LSTM_decoder')

2024-06-03 07:40:01.267025: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-03 07:40:01.343885: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-03 07:40:01.351023: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [10]:
a,b,c = lstm_encoder(datas)

x,_,_ = lstm_decoder(X,initial_state=[b,c])

2024-06-03 07:40:01.924325: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


In [11]:
class Encoder_layer(tf.keras.layers.Layer):
    def __init__(self, num_untis:int):
        super(Encoder_layer,self).__init__()


        self.LSTM_encoder = tf.keras.layers.LSTM(units=num_untis,
                                                 return_state = True,
                                                 name = 'LSTM_encoder')
        
    def call(self,inputs):
        
        output_encoder, state_h, state_c = self.LSTM_encoder(inputs)

        return output_encoder, state_h, state_c
    
teste = Encoder_layer(64)

x,a,b = teste(datas)

In [12]:
class Decoder_layer(tf.keras.layers.Layer):
    def __init__(self, num_untis:int):
        super(Decoder_layer, self).__init__()

        self.LSTM_decoder = tf.keras.layers.LSTM(units=num_untis,
                                                 return_sequences=True,
                                                 return_state = True,
                                                 name = 'LSTM_decoder')

        self.Dense_output = tf.keras.layers.Dense(2)
    def call(self,inputs,encoder_states):


        output_decoder, state_h, state_c = self.LSTM_decoder(inputs,initial_state=encoder_states)
        output = self.Dense_output(output_decoder)

        return output

In [13]:
teste1 = Decoder_layer(64)

x = teste1(X, encoder_states =[a,b])

In [14]:
class Seq_to_Seq_LSTM(tf.keras.Model):
    def __init__(self, num_units:int):

        super().__init__()
        self.num_units = num_units


        self.encoder_layer = Encoder_layer(self.num_units)
        self.decoder_layer = Decoder_layer(self.num_units)


    def call(self,inputs):

        enc,dec= inputs
       
        _, state_h, state_c = self.encoder_layer(enc)

        output_decoder = self.decoder_layer(dec, encoder_states = [state_h, state_c])

        return output_decoder
    




In [16]:

model = Seq_to_Seq_LSTM(num_units=64)

optmin = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optmin, loss='mae')


model.fit([datas,X],
          X,epochs=100,verbose=True,batch_size = 64)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 51.4381
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 51.1079
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 50.8326
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 50.5852
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 50.4192
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 50.2613
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 50.1258
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 49.9955
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 49.8689
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 49.7436
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 49.6160
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 49.4823
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 49.3391
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 49.1844
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 49.0195
Epoch 